In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2024-05-19 12:54:55--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   106MB/s    in 1.5s    

2024-05-19 12:54:56 (106 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os

for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
    print(f"dirpath: {dirpath} has {len(dirnames)} directory which has {len(filenames)}")


dirpath: 10_food_classes_10_percent has 2 directory which has 0
dirpath: 10_food_classes_10_percent/test has 10 directory which has 0
dirpath: 10_food_classes_10_percent/test/sushi has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/chicken_wings has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/grilled_salmon has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/chicken_curry has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/hamburger has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/steak has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/ice_cream has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/fried_rice has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/ramen has 0 directory which has 250
dirpath: 10_food_classes_10_percent/test/pizza has 0 directory which has 250
dirpath: 10_food_classes_10_percent/train has 10 directory

In [ ]:
from tensorflow.keras import backend as K

K.clear_session()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import pandas as pd
import pathlib
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation, Input, Dropout
from tensorflow.keras import Sequential
import matplotlib.image as mpimg
import tensorflow_hub as hub
from tensorflow_hub import KerasLayer

In [ ]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1 / 255.)
test_datagen = ImageDataGenerator(rescale=1 / 255.)

train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=BATCH_SIZE,
                                               target_size=IMAGE_SHAPE,
                                               class_mode="categorical",
                                               seed=42,
                                               shuffle=False)

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             batch_size=BATCH_SIZE,
                                             target_size=IMAGE_SHAPE,
                                             class_mode="categorical",
                                             seed=42, )

class_names = train_data.class_indices
class_names = list(class_names.keys())

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
def tensorboard_callback(experiment_name, dir_name='logs/fit/'):
    log_dir = dir_name + experiment_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    return tensorboard_callback

In [ ]:
effiecientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

In [ ]:
def create_model(model_url, num_classes=10):
    feature_extractor_layer = KerasLayer(model_url,
                                         trainable=False,
                                         input_shape=IMAGE_SHAPE + (3,))
    model = Sequential()

    model.add(feature_extractor_layer)

    model.add(layers.Dense(num_classes, activation="softmax"))

    return model

In [ ]:
ResNet_Model = create_model(resnet_url,
                            num_classes=train_data.num_classes)

In [ ]:
ResNet_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ResNet_Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [ ]:
history_resnet_model = ResNet_Model.fit(train_data,
                                        epochs=10,
                                        steps_per_epoch=len(train_data),
                                        validation_data=test_data,
                                        validation_steps=len(test_data),
                                        callbacks=[tensorboard_callback("History_ResNet_Model")])

Epoch 1/10
24/24 [==============================] - 34s 885ms/step - loss: 3.8250 - accuracy: 0.0520 - val_loss: 2.1784 - val_accuracy: 0.2656
Epoch 2/10
24/24 [==============================] - 17s 748ms/step - loss: 2.1504 - accuracy: 0.2680 - val_loss: 1.4914 - val_accuracy: 0.5164
Epoch 3/10
24/24 [==============================] - 18s 765ms/step - loss: 1.5063 - accuracy: 0.5387 - val_loss: 1.1689 - val_accuracy: 0.6396
Epoch 4/10
24/24 [==============================] - 18s 754ms/step - loss: 1.1211 - accuracy: 0.6373 - val_loss: 0.9917 - val_accuracy: 0.7000
Epoch 5/10
24/24 [==============================] - 19s 827ms/step - loss: 0.8840 - accuracy: 0.7427 - val_loss: 0.9068 - val_accuracy: 0.7272
Epoch 6/10
24/24 [==============================] - 17s 721ms/step - loss: 0.7965 - accuracy: 0.7547 - val_loss: 0.8204 - val_accuracy: 0.7496
Epoch 7/10
24/24 [==============================] - 18s 771ms/step - loss: 0.6634 - accuracy: 0.8080 - val_loss: 0.7751 - val_accuracy: 0.7652

In [ ]:
def preprocessing(filename, img_shape=224):
    # Read the image file
    img = tf.io.read_file(filename)
    # Decode the image; ensure it's converted to three channels (for RGB)
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to the target shape
    img = tf.image.resize(img, size=[img_shape, img_shape])
    # Normalize the image to the [0, 1] range
    img = img / 255.
    return img


In [ ]:
EffiecientNet_Model = create_model(effiecientnet_url,
                            num_classes=train_data.num_classes)

In [ ]:
EffiecientNet_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
EffiecientNet_Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 4062374 (15.50 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 4049564 (15.45 MB)
_________________________________________________________________


In [ ]:
history_effiecientnet_model = EffiecientNet_Model.fit(train_data,
                                        epochs=10,
                                        steps_per_epoch=len(train_data),
                                        validation_data=test_data,
                                        validation_steps=len(test_data),
                                        callbacks=[tensorboard_callback("History_EffiecientNet_Model")])

Epoch 1/10
24/24 [==============================] - 26s 679ms/step - loss: 2.4605 - accuracy: 0.1587 - val_loss: 1.9123 - val_accuracy: 0.4152
Epoch 2/10
24/24 [==============================] - 15s 644ms/step - loss: 1.6975 - accuracy: 0.4680 - val_loss: 1.3701 - val_accuracy: 0.6780
Epoch 3/10
24/24 [==============================] - 14s 604ms/step - loss: 1.2560 - accuracy: 0.7027 - val_loss: 1.0801 - val_accuracy: 0.7676
Epoch 4/10
24/24 [==============================] - 13s 542ms/step - loss: 1.0082 - accuracy: 0.7853 - val_loss: 0.9070 - val_accuracy: 0.8060
Epoch 5/10
24/24 [==============================] - 12s 531ms/step - loss: 0.8791 - accuracy: 0.8067 - val_loss: 0.7963 - val_accuracy: 0.8280
Epoch 6/10
24/24 [==============================] - 13s 562ms/step - loss: 0.7209 - accuracy: 0.8547 - val_loss: 0.7227 - val_accuracy: 0.8436
Epoch 7/10
24/24 [==============================] - 13s 541ms/step - loss: 0.6646 - accuracy: 0.8640 - val_loss: 0.6675 - val_accuracy: 0.8500

In [ ]:
# Assume 'ResNet_Model' is your trained model and 'class_names' is already defined
preprocessed_img = preprocessing("IMG_9559.jpeg")

# Expand dimensions to create a batch of a single image
img_batch = tf.expand_dims(preprocessed_img, axis=0)

# Make predictions
pred = EffiecientNet_Model.predict(img_batch)

# Get the class index with the highest probability
predicted_class_index = tf.argmax(pred[0])

# Retrieve the class name using the index
predicted_class_name = class_names[predicted_class_index]
print(f"Predicted class: {predicted_class_name}")


1/1 [==============================] - 0s 25ms/step
Predicted class: fried_rice
